In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

In [ ]:
P = 3
K = 5

In [ ]:
xs, ys = make_blobs(n_samples=100, centers=P, n_features=2)

In [ ]:
y_idxs = [np.argwhere(ys == y).reshape(-1) for y in np.unique(ys)]

In [ ]:
def plot_blobs(xs, ys, marker='o', include_legend=True):
    for y in np.unique(ys):
        y_idxs = np.argwhere(ys == y).reshape(-1)
        plt.scatter(xs[y_idxs, 0], xs[y_idxs, 1], label=y, marker=marker)
    if include_legend:
        plt.legend()

In [ ]:
plot_blobs(xs, ys)
plt.title('Data points (embeddings)')
plt.show()

In [ ]:
def arranged_idxs_batch(xs, y_idxs, k):
    batch_xs = []
    batch_ys = []
    for y_idx in y_idxs:
        batch_idx = np.random.choice(y_idx, k, replace=False)
        batch_xs.append(xs[batch_idx])
        batch_ys.append(ys[batch_idx])
    batch_xs = np.concatenate(batch_xs, 0)
    batch_ys = np.concatenate(batch_ys)
    return batch_xs, batch_ys

In [ ]:
batch_xs, batch_ys = arranged_idxs_batch(xs, y_idxs, K)

In [ ]:
plot_blobs(xs, ys)
plot_blobs(batch_xs, batch_ys, marker='x', include_legend=False)
plt.title('Data points (embeddings) and selected batch (x)')
plt.show()

In [ ]:
def euclidean_distance(a, b):
    ed = np.sqrt(np.sum(np.square(a[:, np.newaxis, :] - b), axis=2))
    return ed

In [ ]:
ed = euclidean_distance(batch_xs, batch_xs)

In [ ]:
plt.imshow(ed)
plt.title('Distance matrix')
plt.show()

In [ ]:
def batch_hard(axs, ays, p, k):
    distance_matrix = euclidean_distance(axs, axs)
    # matrix side size
    ays = ays.reshape(-1, 1)
    num = ays.shape[0]
    diagonal_idxs = diag_idxs = np.diag_indices(num)
    # mask items where labels are equal
    y_equals = ays == ays.T
    # let's find HARD POSITIVES
    y_equals[diagonal_idxs] = False
    d2p = distance_matrix.copy()
    # we are going to find hard positive - the most distant positive
    # therefore set -infinity distance to the same items
    d2p[y_equals == False] = -np.inf
    p_idxs = np.argmax(d2p, axis=1)
    hard_positives = axs[p_idxs]
    # let's find HARD NEGATIVES
    d2n = distance_matrix.copy()
    y_equals[diagonal_idxs] = True
    d2n[y_equals == True] = np.inf
    n_idxs = np.argmin(d2n, axis=1)
    hard_negatives = axs[n_idxs]
    return axs, hard_positives, hard_negatives

In [ ]:
# shuffle labels and features
#p = np.random.permutation(batch_ys.shape[0])
#batch_xs = batch_xs[p]
#batch_ys = batch_ys[p]
#anchors, hard_positives, hard_negatives = batch_hard(batch_xs, batch_ys, P, K)

In [ ]:
anchors, hard_positives, hard_negatives = batch_hard(batch_xs, batch_ys, P, K)

In [ ]:
EXAMPLES_PER_CLASS = 3
for y in range(P):
    p_idxs = np.argwhere(batch_ys == y).reshape(-1)
    p_anchors = anchors[p_idxs]
    p_hard_positives = hard_positives[p_idxs]
    p_hard_negatives = hard_negatives[p_idxs]
    for j in range(EXAMPLES_PER_CLASS):
        plot_blobs(xs, ys)
        plot_blobs(batch_xs, batch_ys, marker='x', include_legend=False)
        plt.scatter(p_anchors[j, 0], p_anchors[j, 1], marker='*', s=200, label='anchor')
        plt.scatter(p_hard_positives[j, 0], p_hard_positives[j, 1], marker='P', s=200, label='positive')
        plt.scatter(p_hard_negatives[j, 0], p_hard_negatives[j, 1], marker='X', s=200, label='negative')
        plt.title(f'Data points and selected triplet for class {y}')
        plt.legend()
        plt.show()

# TODO

* choose one of the hard positives/negatives randomly to reduce overfitting (because of identical triplets, see identical_triplets.png)